# Login

In [1]:
import tidalapi
import keyring
from plyer import notification
import pandas as pd

session = tidalapi.Session()

session.login_oauth_simple()
print(session.check_login())

Visit https://link.tidal.com/XHEVG to log in, the code will expire in 300 seconds
True


In [2]:
token_type = session.token_type
access_token = session.access_token
refresh_token = session.refresh_token # Not needed if you don't care about refreshing
expiry_time = session.expiry_time

In [3]:
#Load for another device
print(session.load_oauth_session(token_type, access_token, refresh_token, expiry_time))

True


In [4]:
def to_duration_in_minutes(string):
    string = str(string)
    lista = []
    if len(string)<3:
        lista.append('00:'+string)
    else:
        if len(string)%2 !=0:
            string = '0' + string
        while len(string)>0:
            string_tail = string[-2:]
            lista = [string_tail] + lista
            string=string[:-2]
            
    lista = [int(n) for n in lista]     
    lista[0] = lista[0]/60
    try:
        lista[2] = lista[2]*60
    except:
        None
    return lista.sum()

# My favorite...

In [5]:
from IPython.display import display,HTML

def show_image(path,size):
    return f'<img src="{str(path)}" width="{str(size)}">'


def hiperenlace(path):
    return f'<a href="{str(path)}">{str(path)}</a>'

album_cats = ['image',
 'name',
 'artist',
 'artists',
 'year',
 'version',
 'num_tracks',
 'num_volumes',
 'release_date',
 'popularity',
 'copyright',
 'explicit',
 'available_release_date',
 'available']

cats = {'albums':album_cats}

def my_favorite(that,show_images=False):
    favorites = tidalapi.Favorites(session,session.user.id)
    
    list_of_favorites = getattr(favorites,that)(limit=10000)
    
    list_of_atributes = dir(list_of_favorites[0])
    cols = [a for a in list_of_atributes if a[0]!='_']
    to_series = pd.Series(list_of_favorites)
    dicc = {}
    for col in cats[that]:
        if col == 'artist':
            dicc[col] = to_series.apply(lambda x: [getattr(getattr(x,col),'name')])
        elif col == 'artists':
            dicc[col] = to_series.apply(lambda x: list(pd.Series(getattr(x,col)).apply(getattr,args=('name',))))
        elif col == 'image':
            dicc[col] = to_series.apply(lambda x: show_image(getattr(x,col)(1280),60) if show_images==True else hiperenlace(getattr(x,col)(1280)))
        else:
            dicc[col] = to_series.apply(lambda x: getattr(x,col))
    
    df = pd.DataFrame(dicc)
    
    df['artists'] = df.apply(lambda row : ','.join([a for a in row['artists'] if a not in row['artist']]),axis=1)
    df['version'] = df['version'].replace('None','')
    df['artist'] = df['artist'].apply(lambda x : x[0])

    return df
prob = my_favorite('albums',show_images=False)
display(HTML(prob.head(5).to_html(escape=False)))

,image,name,artist,artists,year,version,num_tracks,num_volumes,release_date,popularity,copyright,explicit,available_release_date,available
0,https://resources.tidal.com/images/74cf0ec4/3e21/40bc/95a2/eed029caf776/1280x1280.jpg,( ),Sigur Rós,,2002,None,8,1,2002-10-29,26,© 2002 KRUNK,False,2002-10-29,True
1,https://resources.tidal.com/images/c727e6a3/7bcd/415d/9416/c898dfd1d3ad/1280x1280.jpg,+,La Urss,,2022,None,12,1,2022-09-02,6,Humo Internacional,False,2022-09-02,True
2,https://resources.tidal.com/images/8b4bcc94/b9a7/4b35/9aed/197d46daf723/1280x1280.jpg,"10,000 gecs",100 gecs,,2023,None,10,1,2023-03-17,33,"Dog Show Records/Atlantic Records, © 2023 Atlantic Recording Corporation",True,2023-03-17,True
3,https://resources.tidal.com/images/f5017ab2/b3d1/4b1e/8a4c/7bb1629b51a1/1280x1280.jpg,100% Colombian,Fun Lovin' Criminals,,1998,None,14,1,1998-11-17,14,Chrysalis Records Limited,True,1998-11-17,True
4,https://resources.tidal.com/images/db3d10db/aba5/48ad/ad63/11a008fef8e3/1280x1280.jpg,"100 Days, 100 Nights",Sharon Jones,The Dap-Kings,2007,None,10,1,2007-04-03,26,Daptone Records,False,2007-04-03,True


# Album lists

In [6]:
import requests

url = 'https://pitchfork.com/features/lists-and-guides/best-albums-2023/'

def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        html_text = response.text
        return html_text
    else:
        return "Error en la solicitud:", response.status_code
code = get_html(url)

In [7]:
import re
left_label = '\d{1,2}\.</div><h2>'
right_label = '</em></h2>'
splitter_pos_art = '.</div><h2>'
splitter_art_alb = ': <em>'
regex = fr'{left_label}.*{right_label}'
listado = re.findall(regex,code)
pos = [s.split(splitter_pos_art)[0] for s in listado]
artists = [s.split(splitter_pos_art)[1].split(splitter_art_alb)[0] for s in listado]
albums = [s.split(splitter_art_alb)[1].split(right_label)[0] for s in listado]
df_dic = {'n':pos,'artist':artists,'album':albums}
pitchfork_2023 = pd.DataFrame(df_dic)
pitchfork_2023['n'] = pitchfork_2023['n'].astype(int)
pitchfork_2023.sort_values(by='n')

,n,artist,album
48,1,SZA,SOS
47,2,Caroline Polachek,"Desire, I Want to Turn Into You"
46,3,billy woods / Kenny Segal,Maps
45,4,Wednesday,Rat Saw God
44,5,Nourished by Time,Erotic Probiotic 2
43,6,Sufjan Stevens,Javelin
42,7,Amaarae,Fountain Baby
41,8,Fever Ray,Radical Romantics
40,9,ANOHNI and the Johnsons,My Back Was a Bridge for You to Cross
39,10,yeule,softscars


In [8]:
# The purpose is to get df album ids to add all their tracks to a playlist in Tidal
def get_albums_id(df):
    descripcion = []
    id_upload = []
    for row in df.itertuples():
        s = session.search(query=row[3] + ' ' + row[2])
        aux = False
        for album in s['albums']:
            if album.artist.name == row[2]:
                aux = True
                id_upload.append(album.id)
                break
        if aux == False:
            descripcion.append(row[3] + ' ' + row[2])
    descripcion = ' | '.join(descripcion)
    return {'ids':id_upload,'desc':descripcion}

In [9]:
get_albums_id(pitchfork_2023)

{'ids': [328222087,
  287363526,
  311040940,
  280478079,
  316283554,
  273416583,
  278852722,
  264883317,
  298162719,
  260984185,
  271095786,
  272276481,
  265083968,
  318604951,
  312979383,
  270943364,
  277266388,
  271240948,
  292921818,
  299118996,
  307304924,
  304555200,
  290581948,
  281614588,
  267987548,
  265275765,
  278094339,
  271895787,
  286103660,
  282167416,
  314676437,
  309034503,
  283735625,
  304543910,
  280147270,
  298478256,
  308769818,
  283901370,
  266859518,
  275761405,
  264617506],
 'desc': 'After the Magic 파란노을 (Parannoul) | &amp; the Charm Avalon Emerson | Détwat HiTech | Fly or Die Fly or Die Fly or Die ((world war)) jaimie branch | Love in Exile Arooj Aftab / Vijay Iyer / Shahzad Ismaily | I Killed Your Dog L’Rain | My Back Was a Bridge for You to Cross ANOHNI and the Johnsons | Maps billy woods / Kenny Segal'}

In [10]:
session.search.__doc__

"Searches TIDAL with the specified query, you can also specify what models you\n        want to search for. While you can set the offset, there aren't more than 300\n        items available in a search.\n\n        :param query: The string you want to search for\n        :param models: A list of tidalapi models you want to include in the search.\n            Valid models are :class:`.Artist`, :class:`.Album`, :class:`.Track`, :class:`.Video`, :class:`.Playlist`\n        :param limit: The amount of items you want included, up to 300.\n        :param offset: The index you want to start searching at.\n        :return: Returns a dictionary of the different models, with the dictionary values containing the search results.\n            The dictionary also contains a 'top_hit' result for the most relevant result, limited to the specified types\n        "

In [11]:
s = session.search(query='Let it bleed Rolling Stones')
for album in s['albums']:
    print(album.id,album.year,album.name,album.artist.name,album.num_tracks,album.num_volumes)

121021956 1969 Let It Bleed (50th Anniversary Edition / Remastered 2019) The Rolling Stones 9 1
5693590 1969 Let It Bleed The Rolling Stones 9 1
77695524 1969 Let It Bleed The Rolling Stones 9 1
121021946 1969 Let It Bleed (50th Anniversary Edition / Remastered 2019) The Rolling Stones 9 1
120971269 1969 Let It Bleed (50th Anniversary Edition / Remastered 2019) The Rolling Stones 9 1
